# 결국엔 완성이 된거 같아서 합체 후 작업
3. requirements.csv -> 1번과 2번을 활용한 요구사항 명세서

In [95]:
import pandas as pd
import openai
import requests
import ast
import slack_alarm as alarm

In [57]:
# temperature가 높게 작성을 하게 만들어서 약간 오류가 있긴함
## 해당 파일이 수정된 파일
df = pd.read_csv('test.csv')
len(df)

978

## 978개로 데이터 생성 진행
- AWS에 있는거 활용... 을 하면 좀 그런가
- 아무도 안 쓰는거 같으니까 테스트 겸 한번 해보자
- AWS 주소 : http://13.209.5.218:8000/

In [58]:
df.columns

Index(['subject', 'user_input', 'requirements'], dtype='object')

### 진행방법
1. 요구사항 명세서 추가 생성
2. 프로젝트 요약 생성
3. ERD,API 명세서 생성

In [ ]:
base_url = 'http://13.209.5.218:8000/'

# 요구사항 명세서 생성 함수
def generate_requirements(project_overview, requirements) :
    """
    project_overview : 유저가 입력한 project 데이터(json 형태)
    requirements : 요구사항 명세서(json 형태)
    """
    # 데이터 input
    data = {
        "project_overview": project_overview,
        "existing_requirements": requirements,
        "additional_count": 5,
        "max_tokens": 4000,
        "temperature": 0.3,
        "model": "gpt-4o-mini"
    }

    # 데이터 호출
    response = requests.post(
                            url = base_url +'api/PJA/requirements/generate', 
                            json = data,
                            headers={"Content-Type": "application/json"})
    new_requirements = response.json()

    # 원본 데이터에 추가로 받은 데이터 추가
    add_requirements = new_requirements['requirements']
    total_requirements = str(ast.literal_eval(requirements) + add_requirements)

    return total_requirements

# 프로젝트 요약 생성 함수 
def generate_project_info(project_overview, total_requirements):
    """
    project_overview : 유저가 입력한 project 데이터(json 형태)
    total_requirements : 요구사항 명세서 생성하는 코드(json 형태)
    """

    # 데이터 input
    data = {
        "project_overview": project_overview,
        "requirements": total_requirements,
        "max_tokens": 4000,
        "temperature": 0.3,
        "model": "gpt-4o-mini"
    }
    
    # 데이터 호출
    response = requests.post(
                            url = base_url +'api/PJA/json_Summury/generate', 
                            json = data,
                            headers={"Content-Type": "application/json"})
    project_info = response.json()

    return project_info

# ERDAPI 생성 함수수
def generate_ERDAPI(project_overview, total_requirements, project_info) :
    """
    project_overview : 유저가 입력한 project 데이터(json 형태)
    total_requirements : 요구사항 명세서 생성하는 데이터(json 형태)
    project_info : 프로젝트 요약한 데이터(json 형태태)
    """

    # 데이터 input
    data = {
            "project_overview": project_overview,
            "requirements": total_requirements,
            "project_summury": project_info,
            "max_tokens": 8000,
            "temperature": 0.3,
            "model": "ft:gpt-4o-mini-2024-07-18:test::BebIPMSD"
        }
    
    # 데이터 호출
    response = requests.post(
                        url = base_url +'api/PJA/json_ERDAPI/generate', 
                        json = data,
                        headers={"Content-Type": "application/json"})
    ERDAPI_data = response.json()
    
    return ERDAPI_data


In [ ]:
# 테스트로 10개만
project_overview_lst = df['user_input'][:10]
requirements_lst= df['requirements'][:10]

# 더미로 돌렸을때 걸리는 시간 체크도 필요할듯

# 데이터 저장해놓을 리스트
total_requirements_lst, project_info_lst, ERDAPI_data_lst = [], [], []

for project_overview, requirements, num in zip(project_overview_lst, requirements_lst, range(len(requirements_lst))) :

    # 각 코드 실행 후 리스트에 백업
    total_requirements = generate_requirements(project_overview, requirements)
    total_requirements_lst.append(total_requirements)

    project_info = generate_requirements(project_overview, total_requirements)
    project_info_lst.append(project_info)

    ERDAPI_data = generate_ERDAPI(project_overview, total_requirements, project_info)
    ERDAPI_data_lst.append(ERDAPI_data)

    if num+1 % 50 == 0 :
        alarm.send_slack(f"{num}번째 실행 완료")

alarm.send_slack(f"더미데이터 생성 작업 완료")

True

In [98]:
total_requirements_lst

["[{'requirementType': 'FUNCTIONAL', 'content': '사용자는 회원가입 시 이메일 인증을 통해 계정을 활성화할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '사용자는 스터디 그룹 내에서 퀴즈를 생성하고, 다른 사용자가 생성한 퀴즈에 참여할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '공지사항 게시판은 그룹 내 모든 회원이 읽고 댓글을 남길 수 있어야 하며, 관리자는 공지사항을 삭제할 수 있어야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 사용자 요청에 대해 평균 2초 이내에 응답해야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 동시 사용자 100명 이상을 지원할 수 있어야 하며, 이 경우에도 응답 시간이 3초를 초과하지 않아야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '데이터베이스는 1,000개의 스터디 그룹과 각 그룹당 최대 500개의 퀴즈를 처리할 수 있어야 하며, 데이터 조회는 1초 이내에 완료되어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '사용자는 스터디 그룹 내에서 과제를 생성하고, 다른 사용자와 과제 진행 상황을 공유할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '사용자는 스터디 그룹 내에서 다른 사용자와 직접 메시지를 주고받을 수 있는 기능을 제공해야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 최대 1,000개의 동시 사용자 요청을 처리할 수 있어야 하며, 이 경우에도 평균 응답 시간이 4초를 초과하지 않아야 한다.'}, {'requirementTyp

In [99]:
project_info_lst

["[{'requirementType': 'FUNCTIONAL', 'content': '사용자는 회원가입 시 이메일 인증을 통해 계정을 활성화할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '사용자는 스터디 그룹 내에서 퀴즈를 생성하고, 다른 사용자가 생성한 퀴즈에 참여할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '공지사항 게시판은 그룹 내 모든 회원이 읽고 댓글을 남길 수 있어야 하며, 관리자는 공지사항을 삭제할 수 있어야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 사용자 요청에 대해 평균 2초 이내에 응답해야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 동시 사용자 100명 이상을 지원할 수 있어야 하며, 이 경우에도 응답 시간이 3초를 초과하지 않아야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '데이터베이스는 1,000개의 스터디 그룹과 각 그룹당 최대 500개의 퀴즈를 처리할 수 있어야 하며, 데이터 조회는 1초 이내에 완료되어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '사용자는 스터디 그룹 내에서 과제를 생성하고, 다른 사용자와 과제 진행 상황을 공유할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '사용자는 스터디 그룹 내에서 다른 사용자와 직접 메시지를 주고받을 수 있는 기능을 제공해야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 최대 1,000개의 동시 사용자 요청을 처리할 수 있어야 하며, 이 경우에도 평균 응답 시간이 4초를 초과하지 않아야 한다.'}, {'requirementTyp

In [100]:
ERDAPI_data_lst

[{'detail': "JSON 파싱 오류: Expecting ',' delimiter: line 1 column 4520 (char 4519)"},
 {'detail': "JSON 파싱 오류: Expecting ',' delimiter: line 1 column 3420 (char 3419)"},
 {'detail': "JSON 파싱 오류: Expecting ',' delimiter: line 1 column 3413 (char 3412)"},
 {'detail': "JSON 파싱 오류: Expecting ',' delimiter: line 1 column 580 (char 579)"},
 {'json': {'erd_tables': [{'name': 'EmergencyRoom',
     'erd_columns': [{'name': 'roomId',
       'data_type': 'integer',
       'is_primary_key': True,
       'is_foreign_key': False,
       'is_nullable': False},
      {'name': 'name',
       'data_type': 'string',
       'is_primary_key': False,
       'is_foreign_key': False,
       'is_nullable': False},
      {'name': 'address',
       'data_type': 'string',
       'is_primary_key': False,
       'is_foreign_key': False,
       'is_nullable': False},
      {'name': 'phoneNumber',
       'data_type': 'string',
       'is_primary_key': False,
       'is_foreign_key': False,
       'is_nullable': True},
